# System Dynamics and Equations of Motion

This is the model of the cart. There are two generalized coordinates: $\theta$ (the pendulum angle) and $\phi$ (the wheel angle). Because we assume that all the wheels are rolling without slipping, each wheel has the same angle. Further, the $x$ coordinate of the cart can be expressed in terms of the wheel angle and the radius of the wheels, $r$. Lastly, the input to the system is some torque, $\tau$, that is applied equally to all the wheels.

The first step in the Lagrangian mechanics approach is to calculate the total kinetic energy of the system and the total potential energy of the system with respect to the generalized coordinates and their derivatives. We can do this using the Python package Sympy.

In [ ]:
# Import everything we need from SymPy
import sympy as sym
import numpy as np
from sympy import Symbol, Matrix, Function, Derivative, N
from sympy import diff, simplify, sin, cos, solve, init_printing, symbols, lambdify
init_printing() # This function will make the outputs of SymPy look prettier and be easier to read

Now we are going to define the parameters of the system as constants:

In [ ]:
# Constants of the system
mp = 1.0
mc = 2.0
mw = 0.25
l = 1.0
r = 0.25
g = 9.81

Now we make symbols and functions. These are elements of SymPy and can be thought of as exactly the same as symbols (variables) and functions from math.

In [ ]:
# Time is a symbol (variable)
t = Symbol('t')

# The generalized coordinates and the input torque are both functions of time.
# This means that they are initialized as Functions.
theta = Function('theta')
phi = Function('phi')
tau = Function('tau')

Now we will start to calculate the energies of the system. We can start by getting the kinetic energy of the mass on top of the pendulum. We can do this by:
1. Defining its position in terms of the generalized coordinates
2. Take the derivate of the position with respect to time to get the velocity
3. Calculate the kinetic energy from the velocity
4. Calculate the potential energy by observation

In [ ]:
# Get the position of the mass in terms of phi and theta
pos = Matrix([-r*phi(t) - l*sin( theta(t) ),
              0.0,
              l*cos( theta(t) )])

In [ ]:
# Take the derivative of the position with respect to time
vel = diff(pos,t)

In [ ]:
# Get the kinetic energy of the mass from the velocity
mass_KE = 0.5 * mp * (vel.T @ vel)[0,0]

In [ ]:
# Get the potential energy of the mass based on its position
mass_PE = mp*g*l*cos(theta(t))

Because the connecting rod of the pendulum has no mass nor mass moment of inertia, we can move on the the energy of the cart. The cart does not change height, so we can just set its potential energy to 0. This means all we have to do is calculate the kinetic energy.

In [ ]:
# Get the position of the cart in terms of phi and theta
pos = Matrix([-phi(t)*r,
              0.0,
              0.0])

In [ ]:
# Take the derivative of the position with respect to time
vel = diff(pos,t)

In [ ]:
# Get the kinetic energy of the cart from the velocity
cart_KE = 0.5 * mc * (vel.T @ vel)[0,0]

Next up, the wheels. For the same reason as the cart, we set their potential energy to 0 also. This means we just need to calculate thier kinetic and rotational energy. Also, while calculating the position, we will ignore the constant offsets that each of the four wheels have (length and width of the wheel base). We can do this because constants disappear during differentiation.

In [ ]:
# Get the position of the wheels in terms of phi and theta
pos = Matrix([-phi(t)*r,
              0.0,
              0.0])

In [ ]:
# Take the derivative of the position with respect to time
vel = diff(pos,t)

In [ ]:
# Get the kinetic energy of all four wheels from the velocity
wheel_KE = 4.0 * (0.5 * mw * (vel.T @ vel)[0,0])

In [ ]:
# Get the rotational kinetic energy of all four wheels
I = 0.5*mw*r**2
wheel_RE = 4.0*(I*diff(phi(t),t)**2)

Now we calculate the lagrangian of the system via the formula:$$L=T-V$$ where $T$ is the total kinetic energy and $V$ is the total potential energy.

In [ ]:
# Get the lagrangian
L = (mass_KE + cart_KE + wheel_KE + wheel_RE) - mass_PE
print("System Lagrangian:")
L = simplify(L)
N(L, 3) # This rounds floating point number to 3 places and then prints to the screen

Finally, we get the equations of motion of the system via the formulas:
$$\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\theta}} \right) - \frac{\partial L}{\partial \theta}=0$$
$$\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\phi}} \right) - \frac{\partial L}{\partial \phi}=\tau$$

In [ ]:
# Get the first equation of motion
f1 = diff(diff(L, Derivative(theta(t), t)), t) - diff(L, theta(t))

In [ ]:
# Get the second equation of motion
f2 = diff(diff(L, Derivative(phi(t), t)), t) - diff(L, phi(t)) - 4.0*tau(t)

Currently, the equations of motion are in the form:
$$
f_1 \left( \frac{\partial^{2} \theta}{\partial t^{2}}, \frac{\partial^{2} \phi}{\partial t^{2}}, \frac{\partial \theta}{\partial t}, \frac{\partial \phi}{\partial t}, \theta, \phi, \tau \right) = 0
$$
$$
f_2 \left( \frac{\partial^{2} \theta}{\partial t^{2}}, \frac{\partial^{2} \phi}{\partial t^{2}}, \frac{\partial \theta}{\partial t}, \frac{\partial \phi}{\partial t}, \theta, \phi, \tau \right) = 0
$$

This form of equations of motion is not very helpful to us right now, so next we will place them in the form:
$$
\frac{\partial^{2} \theta}{\partial t^{2}} = f_{\theta} \left( \frac{\partial \theta}{\partial t}, \frac{\partial \phi}{\partial t}, \theta, \phi, \tau \right)
$$
$$
\frac{\partial^{2} \phi}{\partial t^{2}} = f_{\phi} \left( \frac{\partial \theta}{\partial t}, \frac{\partial \phi}{\partial t}, \theta, \phi, \tau \right)
$$

In [ ]:
# Solve the first two equations of motion for the second order derivatives
soln = solve([f1, f2],
             Derivative(theta(t), (t, 2)),
             Derivative(phi(t), (t, 2)))
soln = simplify(soln)
f_theta = soln[Derivative(theta(t), (t, 2))]
f_phi = soln[Derivative(phi(t), (t, 2))]

Let's now combine these two equations into a single vector and replace the functions of time with symbols (this will make the linearization steps easier).

In [ ]:
# Build the equations of motion vector
f = Matrix([f_theta,
            f_phi])

# Replace the functions of time with symbols
(v1, v2, v3, v4, v5) = symbols('v1, v2, v3, v4, v5')
f = f.subs({Derivative(theta(t), t) : v1,
            Derivative(phi(t), t) : v2,
            theta(t) : v3,
            phi(t) : v4,
            tau(t) : v5})
(thetadot, phidot, theta, phi, tau) = symbols('thetadot, phidot, theta, phi, tau')
f = f.subs({v1 : thetadot,
            v2 : phidot,
            v3 : theta,
            v4 : phi,
            v5 : tau})

# Simplify
f = simplify(f)

$f$ is our system model such that:
$$
\begin{bmatrix}
\ddot{\theta} \\
\ddot{\phi} \\
\end{bmatrix} = f(\dot{\theta}, \dot{\phi}, \theta, \phi, \tau)
$$
Note that both of these ODEs are second-order — you will have to replace them each with a set of two first-order ODEs, as usual.

In [ ]:
N(f, 3) # This rounds floating point number to 3 places and then prints to the screen